__This notebook can be used to convert the root files used by the g-2 field team into dataframes__

The libaries loaded below are needed:

In [1]:
import numpy as np
import ROOT
import pickle
import pandas as pd

Welcome to JupyROOT 6.16/00


# 1) Inline C++ functions

## Trolley functions

### Function to get number of entries

In [2]:
%%cpp -d
#include "../RootTreeStructs.h"
using namespace std;
int Entries(){
    int run = 3956;
//#    int probe = 0;
    TFile* filein = new TFile(Form("~/Desktop/FieldPlainRootOutput_0%d_tier1.root",run),"read");
    TDirectory * dTrolley = (TDirectory*)filein->Get("TreeGenTrolley");
    TTree* trolley = (TTree*)dTrolley->Get("trolley");
    int N = trolley->GetEntries();
    filein->Close();
    return N;
}

### Function to get basic trolley info

In [3]:
%%cpp -d
#include "../RootTreeStructs.h"
using namespace std;
void TrolleyInfo(double* f, double* p, double* fid_len, int run, int probe){
//  int run = 3956;
//  int probe = 0;
    TFile* filein = new TFile(Form("~/Desktop/FieldPlainRootOutput_0%d_tier1.root",run),"read");
    TDirectory * dTrolley = (TDirectory*)filein->Get("TreeGenTrolley");
    TTree* trolley = (TTree*)dTrolley->Get("trolley");
    gm2field::trolleyTimeStamp_t trolleyTimeStamp;
    gm2field::trolleyProbeFrequency_t trolleyProbeFrequency;
    gm2field::trolleyFieldMultipole_t trolleyFieldMultipole;
    gm2field::trolleyPosition_t trolleyPosition;
    trolley->SetBranchAddress("TimeStamp",&trolleyTimeStamp.GpsCycleStart);
    trolley->SetBranchAddress("ProbeFrequency",&trolleyProbeFrequency.Frequency);
    trolley->SetBranchAddress("FieldMultipole",&trolleyFieldMultipole.Multipole);
    trolley->SetBranchAddress("Position",&trolleyPosition.Phi);
    
    //#for FID length
    gm2field::trolleyProbeSignal_t trolleyProbeSignal;
    //#need to set the first one in the struct, which happens to be amplitude
    trolley->SetBranchAddress("ProbeSignal",&trolleyProbeSignal.Amplitude);
    
///////////////////////////////////////////////////////////////////////////////////
    int nAllReadings = trolley->GetEntries();
    for(int i=0;i<nAllReadings;i++){ //loop through trolley events
        trolley->GetEntry(i);
        f[i] = trolleyProbeFrequency.Frequency[probe][0]; 
        p[i] = trolleyPosition.Phi[probe][2];
        fid_len[i] = trolleyProbeSignal.FidLength[probe];
    }
    filein->Close();
    return;
}

### Seperate function to get trolley FIDs

FID signals are heavy, so a seperate function is made, enabeling the user to select only the desired signals

In [4]:
%%cpp -d
#include "../RootTreeStructs.h"
using namespace std;
void RawFid(double* raw_fid,  int run, int reading, int probe){
//  int run = 3956;
//  int probe = 0;
    TFile* filein = new TFile(Form("~/Desktop/FieldPlainRootOutput_0%d_tier1.root",run),"read");
    TDirectory * dMyTrolley = (TDirectory*)filein->Get("TrolleyWfExtraction");
    TTree* myTrolley = (TTree*)dMyTrolley->Get("trolleyWf");
    gm2field::trolleyFid_t trolleyFid;
    myTrolley->SetBranchAddress("trolleyFid", &trolleyFid.RawFid);
///////////////////////////////////////////////////////////////////////////////////
    int fidLength = 16000;
    int dataStartingPoint = probe*fidLength;    
    myTrolley->GetEntry(reading);                                
    for (int i=0; i<fidLength; i++) {
        raw_fid[i] = trolleyFid.RawFid[i];
    }
    filein->Close();
    return;
}

In [5]:
# Loop over all trolley probes for a given run list. 



# 2)  Calling the functions and pickeling the data 

## Converting basic trolley data

* all info for each probe gets stored as a df.
* each probe df gets stored into a dict
* each dict is stored as file named by its run


In [6]:
#set up for the loop
n_entries = ROOT.Entries()
probe_list = [p for p in range(3)]
run_list = [3956]
all_data_frames = {}

for run in run_list:
    print("starting run",run)
    for probe in probe_list:
        print("starting probe",probe)
        freq = np.empty(n_entries)
        pos = np.empty(n_entries)
        fid_len = np.empty(n_entries)
        ROOT.TrolleyInfo(freq,pos,fid_len,run,probe)
        df = pd.DataFrame({'Frequency':freq, 
                           'Position':pos, 
                           'Fid_len': fid_len})
        all_data_frames[probe] = df
        
    with open("./output/tr_basic_" + str(run) + ".pkl", "wb") as f:
        pickle.dump(all_data_frames, f)

starting run 3956
starting probe 0
starting probe 1
starting probe 2


## Converting trolley FIDs

* The user selects specific FIDs 
    - Selection should be based on the user's analysis of the basic trolley data
* The specified fid info is then added to the above-created pkl file



In [7]:
#Choose the data you want by modifying below and reading in
event_list = [4,7,22,30] #user choice
probe = 0 #user choice
run = 3956 #user choice
file = "output/tr_basic_"+str(run)+".pkl"

In [ ]:
#open the file to get a dict of dataframes
with open(file, "rb") as f:
    trolley = pickle.load(f)
#for the chosen probe's dataframe, add a raw FID column
#need pre-formateding because an NP array will be written to each cell
trolley[probe]['RawFid'] = ""
trolley[probe]['RawFid'].astype(object,copy=False)

#write each chosen event's FID to the chosen probe's dataframe
for event in event_list:
    raw_fid = np.empty(16000)
    ROOT.RawFid(raw_fid,run,event,probe)
    trolley[probe].at[event, 'RawFid'] = raw_fid 

In [ ]:
#write over the existing file
with open(file, "wb") as f:
    pickle.dump(trolley, f)

# 3) Instructions on how to load the pkl files elsewhere

To load the pkl files, you would do something like the following, wherein a run's pkl file, containing a dict, is loaded into "trolley"


```python
with open("output/TrolleyInfo_3956.pkl", "rb") as f:
    trolley = pickle.load(f)
```
to get the df for probe 6, for example, simply do

```python
trolley[6]
```

in general, not all probes may be stored. to get a list of probes, just do

```python
keys_list = [key for key in trolley.keys()]
```



